In [1]:
#%matplotlib inline
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
##“The Instacart Online Grocery Shopping Dataset 2017”, Accessed from https://www.instacart.com/datasets/grocery-shopping-2017 on <09-16-2017>

import pyspark.sql.functions as F

This Dataset is from the Instacart Kaggle Competition. The challenge is to use this anonymized data on customer orders over time to predict which previously purchased products will be in a user’s next order. 

It was informed that this dataset is a relational set of 6 files describing customers' orders over time. 

We will start by reading all of them:

In [2]:
aisles = spark.read \
    .options(header=True, inferSchema=True) \
    .csv("Instacart Kaggle/aisles.csv")
aisles.show()

+--------+--------------------+
|aisle_id|               aisle|
+--------+--------------------+
|       1|prepared soups sa...|
|       2|   specialty cheeses|
|       3| energy granola bars|
|       4|       instant foods|
|       5|marinades meat pr...|
|       6|               other|
|       7|       packaged meat|
|       8|     bakery desserts|
|       9|         pasta sauce|
|      10|    kitchen supplies|
|      11|    cold flu allergy|
|      12|         fresh pasta|
|      13|      prepared meals|
|      14|tofu meat alterna...|
|      15|    packaged seafood|
|      16|         fresh herbs|
|      17|  baking ingredients|
|      18|bulk dried fruits...|
|      19|       oils vinegars|
|      20|        oral hygiene|
+--------+--------------------+
only showing top 20 rows



In [3]:
dptmts = spark.read \
    .options(header=True, inferSchema=True) \
    .csv("Instacart Kaggle/departments.csv")
dptmts.show()

+-------------+---------------+
|department_id|     department|
+-------------+---------------+
|            1|         frozen|
|            2|          other|
|            3|         bakery|
|            4|        produce|
|            5|        alcohol|
|            6|  international|
|            7|      beverages|
|            8|           pets|
|            9|dry goods pasta|
|           10|           bulk|
|           11|  personal care|
|           12|   meat seafood|
|           13|         pantry|
|           14|      breakfast|
|           15|   canned goods|
|           16|     dairy eggs|
|           17|      household|
|           18|         babies|
|           19|         snacks|
|           20|           deli|
+-------------+---------------+
only showing top 20 rows



In [4]:
prod_in_orders = spark.read \
    .options(header=True, inferSchema=True) \
    .csv("Instacart Kaggle/order_products__prior.csv")
prod_in_orders.show()

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       2|     33120|                1|        1|
|       2|     28985|                2|        1|
|       2|      9327|                3|        0|
|       2|     45918|                4|        1|
|       2|     30035|                5|        0|
|       2|     17794|                6|        1|
|       2|     40141|                7|        1|
|       2|      1819|                8|        1|
|       2|     43668|                9|        0|
|       3|     33754|                1|        1|
|       3|     24838|                2|        1|
|       3|     17704|                3|        1|
|       3|     21903|                4|        1|
|       3|     17668|                5|        1|
|       3|     46667|                6|        1|
|       3|     17461|                7|        1|
|       3|     32665|                8|        1|


In [5]:
all_orders = spark.read \
    .options(header=True, inferSchema=True) \
    .csv("Instacart Kaggle/orders.csv")
all_orders.show()

+--------+-------+--------+------------+---------+-----------------+----------------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
| 2539329|      1|   prior|           1|        2|                8|                  null|
| 2398795|      1|   prior|           2|        3|                7|                  15.0|
|  473747|      1|   prior|           3|        3|               12|                  21.0|
| 2254736|      1|   prior|           4|        4|                7|                  29.0|
|  431534|      1|   prior|           5|        4|               15|                  28.0|
| 3367565|      1|   prior|           6|        2|                7|                  19.0|
|  550135|      1|   prior|           7|        1|                9|                  20.0|
| 3108588|      1|   prior|           8|        1|               14|            

In [6]:
train = spark.read \
    .options(header=True, inferSchema=True) \
    .csv("Instacart Kaggle/order_products__train.csv")
train.show()

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       1|     49302|                1|        1|
|       1|     11109|                2|        1|
|       1|     10246|                3|        0|
|       1|     49683|                4|        0|
|       1|     43633|                5|        1|
|       1|     13176|                6|        0|
|       1|     47209|                7|        0|
|       1|     22035|                8|        1|
|      36|     39612|                1|        0|
|      36|     19660|                2|        1|
|      36|     49235|                3|        0|
|      36|     43086|                4|        1|
|      36|     46620|                5|        1|
|      36|     34497|                6|        1|
|      36|     48679|                7|        1|
|      36|     46979|                8|        1|
|      38|     11913|                1|        0|


In [7]:
products = spark.read \
    .options(header=True, inferSchema=True) \
    .csv("Instacart Kaggle/products.csv")
products.show()

+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
|         1|Chocolate Sandwic...|      61|           19|
|         2|    All-Seasons Salt|     104|           13|
|         3|Robust Golden Uns...|      94|            7|
|         4|Smart Ones Classi...|      38|            1|
|         5|Green Chile Anyti...|       5|           13|
|         6|        Dry Nose Oil|      11|           11|
|         7|Pure Coconut Wate...|      98|            7|
|         8|Cut Russet Potato...|     116|            1|
|         9|Light Strawberry ...|     120|           16|
|        10|Sparkling Orange ...|     115|            7|
|        11|   Peach Mango Juice|      31|            7|
|        12|Chocolate Fudge L...|     119|            1|
|        13|   Saline Nasal Mist|      11|           11|
|        14|Fresh Scent Dishw...|      74|           17|
|        15|Overnight Diapers..

In [8]:
sample_sub = spark.read \
    .options(header=True, inferSchema=True) \
    .csv("Instacart Kaggle/sample_submission.csv")
sample_sub.show()

+--------+-----------+
|order_id|   products|
+--------+-----------+
|      17|39276 29259|
|      34|39276 29259|
|     137|39276 29259|
|     182|39276 29259|
|     257|39276 29259|
|     313|39276 29259|
|     353|39276 29259|
|     386|39276 29259|
|     414|39276 29259|
|     418|39276 29259|
|     437|39276 29259|
|     452|39276 29259|
|     474|39276 29259|
|     492|39276 29259|
|     497|39276 29259|
|     513|39276 29259|
|     517|39276 29259|
|     604|39276 29259|
|     680|39276 29259|
|     758|39276 29259|
+--------+-----------+
only showing top 20 rows



That was smooth!!! 

Taking a quick look we can have an idea what the data is about.

Are the columns names consistents?
Are there any missing values?

In [9]:
aisles.dtypes

[('aisle_id', 'int'), ('aisle', 'string')]

In [10]:
aisles.describe().show()

+-------+-----------------+--------------------+
|summary|         aisle_id|               aisle|
+-------+-----------------+--------------------+
|  count|              134|                 134|
|   mean|             67.5|                null|
| stddev|38.82653731663435|                null|
|    min|                1|air fresheners ca...|
|    max|              134|              yogurt|
+-------+-----------------+--------------------+



In [11]:
dptmts.dtypes

[('department_id', 'int'), ('department', 'string')]

In [12]:
dptmts.describe().show()

+-------+------------------+----------+
|summary|     department_id|department|
+-------+------------------+----------+
|  count|                21|        21|
|   mean|              11.0|      null|
| stddev|6.2048368229954285|      null|
|    min|                 1|   alcohol|
|    max|                21|    snacks|
+-------+------------------+----------+



In [13]:
prod_in_orders.dtypes

[('order_id', 'int'),
 ('product_id', 'int'),
 ('add_to_cart_order', 'int'),
 ('reordered', 'int')]

In [14]:
prod_in_orders.describe().show()

+-------+------------------+------------------+-----------------+-------------------+
|summary|          order_id|        product_id|add_to_cart_order|          reordered|
+-------+------------------+------------------+-----------------+-------------------+
|  count|          32434489|          32434489|         32434489|           32434489|
|   mean|1710748.5189427834|25576.337535424096|8.351075517175559| 0.5896974667922161|
| stddev|  987300.696452965|14096.689090257634|7.126671161204477|0.49188857672328484|
|    min|                 2|                 1|                1|                  0|
|    max|           3421083|             49688|              145|                  1|
+-------+------------------+------------------+-----------------+-------------------+



In [15]:
products.dtypes

[('product_id', 'int'),
 ('product_name', 'string'),
 ('aisle_id', 'string'),
 ('department_id', 'string')]

In [16]:
products.describe().show()

+-------+------------------+--------------------+-----------------+------------------+
|summary|        product_id|        product_name|         aisle_id|     department_id|
+-------+------------------+--------------------+-----------------+------------------+
|  count|             49688|               49688|            49688|             49688|
|   mean|           24844.5|              1493.0|67.76919516171232|11.728580916537524|
| stddev|14343.834424588147|   620.4329133758138|38.31645086851397| 5.850420587359203|
|    min|                 1|"""Constant Comme...|          Blunted|              Red"|
|    max|             49688|with a Splash of ...|               99|                 9|
+-------+------------------+--------------------+-----------------+------------------+



In [17]:
all_orders.dtypes

[('order_id', 'int'),
 ('user_id', 'int'),
 ('eval_set', 'string'),
 ('order_number', 'int'),
 ('order_dow', 'int'),
 ('order_hour_of_day', 'int'),
 ('days_since_prior_order', 'double')]

In [18]:
all_orders.describe().show()

+-------+-----------------+------------------+--------+------------------+------------------+-----------------+----------------------+
|summary|         order_id|           user_id|eval_set|      order_number|         order_dow|order_hour_of_day|days_since_prior_order|
+-------+-----------------+------------------+--------+------------------+------------------+-----------------+----------------------+
|  count|          3421083|           3421083| 3421083|           3421083|           3421083|          3421083|               3214874|
|   mean|        1710542.0|102978.20805926077|    null|17.154857979183785|2.7762191095626734|13.45201534134074|    11.114836226863012|
| stddev|987581.7398225827| 59533.71779350369|    null|17.733164470966642|2.0468291939879717|4.226088402102022|      9.20673651753377|
|    min|                1|                 1|   prior|                 1|                 0|                0|                   0.0|
|    max|          3421083|            206209|   train|

In [19]:
train.dtypes

[('order_id', 'int'),
 ('product_id', 'int'),
 ('add_to_cart_order', 'int'),
 ('reordered', 'int')]

In [20]:
#The first join will connect the products tables
prod_full = products.join(aisles, on='aisle_id')
prod_full = prod_full.join(dptmts, on='department_id')
prod_full.show(10)

+-------------+--------+----------+--------------------+--------------------+-------------+
|department_id|aisle_id|product_id|        product_name|               aisle|   department|
+-------------+--------+----------+--------------------+--------------------+-------------+
|           19|      61|         1|Chocolate Sandwic...|       cookies cakes|       snacks|
|           13|     104|         2|    All-Seasons Salt|   spices seasonings|       pantry|
|            7|      94|         3|Robust Golden Uns...|                 tea|    beverages|
|            1|      38|         4|Smart Ones Classi...|        frozen meals|       frozen|
|           13|       5|         5|Green Chile Anyti...|marinades meat pr...|       pantry|
|           11|      11|         6|        Dry Nose Oil|    cold flu allergy|personal care|
|            7|      98|         7|Pure Coconut Wate...|       juice nectars|    beverages|
|            1|     116|         8|Cut Russet Potato...|      frozen produce|   

In [21]:
#The second join will connect the orders tables
order_full = prod_in_orders.join(all_orders, on='order_id')

In [22]:
# The third join will connect products and orders
order_prod_full = order_full.join(prod_full, on='product_id')
order_prod_full.describe().show(5)

+-------+------------------+------------------+-----------------+------------------+------------------+--------+------------------+------------------+------------------+----------------------+-----------------+------------------+--------------------+--------------------+----------+
|summary|        product_id|          order_id|add_to_cart_order|         reordered|           user_id|eval_set|      order_number|         order_dow| order_hour_of_day|days_since_prior_order|    department_id|          aisle_id|        product_name|               aisle|department|
+-------+------------------+------------------+-----------------+------------------+------------------+--------+------------------+------------------+------------------+----------------------+-----------------+------------------+--------------------+--------------------+----------+
|  count|          32434486|          32434486|         32434486|          32434486|          32434486|32434486|          32434486|          32434486| 

In [23]:
#Let´s see which are the most relevant departments
o = order_prod_full
o = o.groupby('department').count()
from pyspark.sql.functions import desc
o.sort(desc("count")).show()

+---------------+-------+
|     department|  count|
+---------------+-------+
|        produce|9479291|
|     dairy eggs|5414016|
|         snacks|2887550|
|      beverages|2690129|
|         frozen|2236432|
|         pantry|1875577|
|         bakery|1176787|
|   canned goods|1068058|
|           deli|1051249|
|dry goods pasta| 866627|
|      household| 738663|
|      breakfast| 709569|
|   meat seafood| 708931|
|  personal care| 447123|
|         babies| 423802|
|  international| 269253|
|        alcohol| 153696|
|           pets|  97724|
|        missing|  69145|
|          other|  36291|
+---------------+-------+
only showing top 20 rows



In [24]:
o.d = o.sort(desc("count"))

#Preparar os dados em dataframes para gerar os grafos: user_id & department_id; user_id & aisle_id; user_id & product_id

In [25]:
#drop columns para ficar apenas aisle e user
o = order_prod_full
o = o.drop('product_id')
o = o.drop('order_id')
o = o.drop('order_dow')
o = o.drop('order_hour_of_day')
o = o.drop('department_id')
o = o.drop('product_name')
o = o.drop('aisle')
o = o.drop('department')
o.show(4)

+-----------------+---------+-------+--------+------------+----------------------+--------+
|add_to_cart_order|reordered|user_id|eval_set|order_number|days_since_prior_order|aisle_id|
+-----------------+---------+-------+--------+------------+----------------------+--------+
|               26|        0|  47136|   prior|           3|                  30.0|      24|
|               15|        1|  12428|   prior|          14|                   7.0|      24|
|               10|        0|  63587|   prior|           1|                  null|      24|
|               11|        0| 158452|   prior|          11|                  13.0|      24|
+-----------------+---------+-------+--------+------------+----------------------+--------+
only showing top 4 rows



In [26]:
#drop columns pra ficar apenas departments e user
r = order_prod_full
r = r.drop('product_id')
r = r.drop('order_id')
r = r.drop('order_dow')
r = r.drop('order_hour_of_day')
r = r.drop('aisle_id')
r = r.drop('product_name')
r = r.drop('aisle')
r = r.drop('department')
r.show(4)

+-----------------+---------+-------+--------+------------+----------------------+-------------+
|add_to_cart_order|reordered|user_id|eval_set|order_number|days_since_prior_order|department_id|
+-----------------+---------+-------+--------+------------+----------------------+-------------+
|               26|        0|  47136|   prior|           3|                  30.0|            4|
|               15|        1|  12428|   prior|          14|                   7.0|            4|
|               10|        0|  63587|   prior|           1|                  null|            4|
|               11|        0| 158452|   prior|          11|                  13.0|            4|
+-----------------+---------+-------+--------+------------+----------------------+-------------+
only showing top 4 rows



In [27]:
o = o.groupby(['user_id', 'aisle_id']).count()
o.describe().show()

+-------+-----------------+-----------------+------------------+
|summary|          user_id|         aisle_id|             count|
+-------+-----------------+-----------------+------------------+
|  count|          5729249|          5729249|           5729249|
|   mean|103040.3949648549|70.60027832618202|  5.66121074507322|
| stddev|59464.05655130876|38.16683566799095|12.135918176971554|
|    min|                1|                1|                 1|
|    max|           206209|               99|              1522|
+-------+-----------------+-----------------+------------------+



In [28]:
r = r.groupby(['user_id', 'department_id']).count()
r.describe().show()

+-------+-----------------+------------------+------------------+
|summary|          user_id|     department_id|             count|
+-------+-----------------+------------------+------------------+
|  count|          2232789|           2232789|           2232789|
|   mean|103057.9642895052|11.145940794226414| 14.52644472899141|
| stddev|59504.68954829935| 6.033196795119511|31.551505253745397|
|    min|                1|                 1|                 1|
|    max|           206209|                 9|              1559|
+-------+-----------------+------------------+------------------+



In [29]:
o.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("user_aisle.csv")

In [30]:
r.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("user_department.csv")